# Representações Distribuídas de Texto e Modelagem de Tópicos

<b> DOCENTES: </b>
<pre>
     Prof. Renato Rocha de Souza
     Prof. Mauricio Barcellos Almeida
     Profa. Renata Maria Abrantes Baracho Porto
</pre>

<b> DISCENTE: </b> Jeanne Louize Emygdio </br>

<b> OBJETIVO: </b> Identificar artigos científicos disponíveis na base Medline voltados à temática de uso do padrão OpenEHR - Uma ontologia para modelagem de dados médicos e que possam compor meu acervo particular de referencial teórico para a escrita da tese. 

<b> Estratégia selecionada: </b>  Modelagem de tópicos e clusterização de documentos. 

<b> Etapas: </b>
<pre>
    1.	Finalização do curso de Python disponibilizado pelo Prof. Renato
    2.	Instalação e configuração da plataforma de desenvolvimento
    3.	Fork da área de MMD do Prof. Renato no Github
    4.	Estudo do notebook 4.2_TopicModeling_Clustering_Documents.ipynb
    5.	Importação das bibliotecas necessárias
    6.	Desenvolvimento do novo notebook de acordo com as passagens previstas no 4.2.
</pre>

<b>Resultados esperados: </b>
<pre>
	- Desenvolvimento de habilidades de programação na linguagem Python;
	- Melhor compreensão das estratégias de obtenção de documentos, análises textuais e clusterização;
	- Montagem de biblioteca de artigos relevantes na temática selecionada de forma ágil.
 </pre>

In [25]:
# urllib - Módulo que reúne funções de alto nível para acesso à internet. 
# Documentation: https://docs.python.org/3/library/urllib.html | 
#                https://docs.python.org/3/library/urllib.request.html#module-urllib.request
import urllib as url                 

# BeautifulSoup - Biblioteca utilizada para extrair dados de arquivos HTML e XML.
# Documentation: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
from bs4 import BeautifulSoup as bs

# NLTK - Biblioteca para trabalhar com processamentos de linguagem natural
# Documentation: https://pypi.python.org/pypi/nltk
import nltk
nltk.download('punkt')
from nltk.corpus       import stopwords
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize     import word_tokenize

# Biblioteca para auxílio à contagem de freqência de ocorrência de palavras em um texto
from collections import Counter

# Bilioteca para lidar com processamento de strings através de expressões regulares
import re 

# Biblioteca que disponibiliza recursos para modelagem de tópicos
import gensim

#from selenium import webdriver

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Louize\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Louize\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
# Definindo caminho para saída de arquivos gerados (se houver)
outputs = "c:/outputs/"

In [30]:
# Acessando a página do Portal da Capes. O objetivo é pesquisar a base PUBMED.
# Realizei acesso com e sem autenticação e o resultado da pesquisa foi o mesmo. A codificação segue sem autenticação.
# O link alvo é: https://www.ncbi.nlm.nih.gov/pubmed/
# Termo pesquisado: OpenEHR. Encontrei 148 artigos.
# Outros termos para pesquisa futura: (((ontology[Title]) OR ontology[Abstract]) AND (owl OR 'web ontology' OR 'web ontologies') 

# Lógica utilizada descrita por linhas: Recuperar conteúdo da página usando urllib, e visualizar resultados tratados 
#                                       pela biblioteca BeautifulSoup.
# L20: Estrutura de controle para tratar adequadamente possíveis erros na execução dos comandos (try...except)
# L21: Variável v_terbus recebe a string que vou usar para buscar o termo OpenEHR no título e abstract
# L22: Acesso à url completa da página acrescida do conteúdo da variável v_terbus, utilizando a função urlib
# L23: Execução da biblioteca BeautifulSoup sobre a Url acessada para estudar os elementos da página e extrair os 
#      desejados. O resultado é armazenado na variável v_soup. O comando print(v_soup.prettify()) permite ver toda 
#      a estrutura da página armazenado na variável.
# L24: Declaração do vetor v_titles para armazenamento dos títulos encontrados
# L27 a 29: Após conhecimento dos elementos, busquei as classes que tivessem o valor title e que estivessem dentro de uma 
#      tag <p> (parágrafo) e, em seus resultados, busquei os conteúdos das tags <a>, que me levariam aos títulos dos 
#      artigos. Estruturei dois loops para analisar os dois resultados aninhados. E em seguida armazenei cada título no 
#      vetor v_titles.

try:   
   v_terbus = '(OpenEHR[Title]+OR+OpenEHR[Abstract])' 
   v_url    = url.request.urlopen('https://www.ncbi.nlm.nih.gov/pubmed/?term='+v_terbus) 
   v_soup   = bs(v_url,'html.parser')
   v_titles = [] # Declaração de um vetor para armazenar os títulos encontrados
                                                    
   for v_elemento in v_soup.find_all('p', class_='title'):                 
      for n in bs(v_elemento.find('a').text): # Tentando extrair apenas a parte textual de cada trecho html
          v_titles.append(n)        
except url.error:
   print('Falha na conexão!')
   exit(1) #interrompe  a execução do programa              
print(v_titles)

[<html><body><p>Special Topic Interoperability and EHR: Combining openEHR, SNOMED, IHE, and Continua as approaches to interoperability on national eHealth.</p></body></html>, <html><body><p>Bridging the Gap Between the Social and the Technical: The Enrolment of Socio-Technical Information Architects to Cope with the Two-Level Model of EPR Systems.</p></body></html>, <html><body><p>Domain Modeling and Application Development of an Archetype- and XML-based EHRS. Practical Experiences and Lessons Learnt.</p></body></html>, <html><body><p>Applying an Archetype-Based Approach to Electroencephalography/Event-Related Potential Experiments in the EEGBase Resource.</p></body></html>, <html><body><p>Querying Archetype-Based Electronic Health Records Using Hadoop and Dewey Encoding of openEHR Models.</p></body></html>, <html><body><p>Introducing a Method for Transformation of Paper-Based Research Data into Concept-Based Representation with openEHR.</p></body></html>, <html><body><p>Integration of

C:\Users\Louize\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Louize\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [31]:
# Conferindo quantos títulos de artigos foram armazenados no vetor v_title
# Problema encontrado: não consegui ainda fazer o sistema localizar os 148 artigos, ele só está considerando os que 
# foram exibidos na primeira página da busca (20 artigos).
print("Number of documents:",len(v_titles))

Number of documents: 20


In [32]:
# Visualizando os artigos armazenados
# Comentário: Não consegui ainda remover as tags com BeautifulSoup, então utilizei a biblioteca 're' e funcionou!
#             Este código deverá ser otimizado no futuro
v_count = 0
while v_count <= 19:
   v_titles[v_count] = (re.sub('<[^>]+?>', '', str(v_titles[v_count])))
   print(v_titles[v_count])
   v_count += 1

Special Topic Interoperability and EHR: Combining openEHR, SNOMED, IHE, and Continua as approaches to interoperability on national eHealth.
Bridging the Gap Between the Social and the Technical: The Enrolment of Socio-Technical Information Architects to Cope with the Two-Level Model of EPR Systems.
Domain Modeling and Application Development of an Archetype- and XML-based EHRS. Practical Experiences and Lessons Learnt.
Applying an Archetype-Based Approach to Electroencephalography/Event-Related Potential Experiments in the EEGBase Resource.
Querying Archetype-Based Electronic Health Records Using Hadoop and Dewey Encoding of openEHR Models.
Introducing a Method for Transformation of Paper-Based Research Data into Concept-Based Representation with openEHR.
Integration of Hospital Information and Clinical Decision Support Systems to Enable the Reuse of Electronic Health Record Data.
Structuring Legacy Pathology Reports by openEHR Archetypes to Enable Semantic Querying.
A Survey of Standa

In [33]:
# É necessário realizar uma limpeza no texto antes de separar os tokens
# Utilizei stopwords com base de palavras em inglês
v_stopwords   = nltk.corpus.stopwords.words('english')

def clean(doc):
    stop_free    = " ".join([i for i in doc.lower().split() if i not in v_stopwords])
    return stop_free

doc_clean = [clean(doc).split() for doc in v_titles]  # A função splif() separa os tokens? (pesquisar)
print(doc_clean)

[['special', 'topic', 'interoperability', 'ehr:', 'combining', 'openehr,', 'snomed,', 'ihe,', 'continua', 'approaches', 'interoperability', 'national', 'ehealth.'], ['bridging', 'gap', 'social', 'technical:', 'enrolment', 'socio-technical', 'information', 'architects', 'cope', 'two-level', 'model', 'epr', 'systems.'], ['domain', 'modeling', 'application', 'development', 'archetype-', 'xml-based', 'ehrs.', 'practical', 'experiences', 'lessons', 'learnt.'], ['applying', 'archetype-based', 'approach', 'electroencephalography/event-related', 'potential', 'experiments', 'eegbase', 'resource.'], ['querying', 'archetype-based', 'electronic', 'health', 'records', 'using', 'hadoop', 'dewey', 'encoding', 'openehr', 'models.'], ['introducing', 'method', 'transformation', 'paper-based', 'research', 'data', 'concept-based', 'representation', 'openehr.'], ['integration', 'hospital', 'information', 'clinical', 'decision', 'support', 'systems', 'enable', 'reuse', 'electronic', 'health', 'record', 'dat

In [34]:
# Criação de um dicionário da lista dos documentos encontrados, baseado em seus títulos
# O dicionário atribui um número a cada termo encontrado. Este valor é útil para que se possa conferir a frequência de 
# ocorrência destes termos nos documentos.
v_dictionary = gensim.corpora.Dictionary(doc_clean)
print(v_dictionary[4])
print(v_dictionary.token2id['openehr'])
print("Number of words in dictionary:",len(v_dictionary))

for i in range(len(v_dictionary)):
    print(i, v_dictionary[i])

combining
52
Number of words in dictionary: 176
0 special
1 topic
2 interoperability
3 ehr:
4 combining
5 openehr,
6 snomed,
7 ihe,
8 continua
9 approaches
10 national
11 ehealth.
12 bridging
13 gap
14 social
15 technical:
16 enrolment
17 socio-technical
18 information
19 architects
20 cope
21 two-level
22 model
23 epr
24 systems.
25 domain
26 modeling
27 application
28 development
29 archetype-
30 xml-based
31 ehrs.
32 practical
33 experiences
34 lessons
35 learnt.
36 applying
37 archetype-based
38 approach
39 electroencephalography/event-related
40 potential
41 experiments
42 eegbase
43 resource.
44 querying
45 electronic
46 health
47 records
48 using
49 hadoop
50 dewey
51 encoding
52 openehr
53 models.
54 introducing
55 method
56 transformation
57 paper-based
58 research
59 data
60 concept-based
61 representation
62 openehr.
63 integration
64 hospital
65 clinical
66 decision
67 support
68 systems
69 enable
70 reuse
71 record
72 data.
73 structuring
74 legacy
75 pathology
76 reports


In [35]:
# Verificando a ocorrência dos termos identificados no dicionário (v_dictionary) em cada documento listado (doc_clean)
v_corpus = [v_dictionary.doc2bow(gen_doc) for gen_doc in doc_clean] 
for d in v_corpus:
    print(d)

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]
[(12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)]
[(25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)]
[(36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1)]
[(37, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1)]
[(54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1)]
[(18, 1), (45, 1), (46, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1)]
[(52, 1), (69, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1)]
[(18, 1), (24, 1), (65, 1), (66, 1), (67, 1), (80, 1), (81, 1), (82, 1)]
[(36, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1)]
[(59, 1), (72, 1), (96, 1), 

In [36]:
# Criação de um modelo de Tópicos Gensim para o Corpus criado (matrix TF-IDF)
# Num_docs = Número de documentos analisados; num_nnz = Número de frequências de termos processadas

tf_idf = gensim.models.TfidfModel(v_corpus)
print(tf_idf)
s = 0
for i in v_corpus:
    s += len(i)
print(s)

TfidfModel(num_docs=20, num_nnz=224)
224


In [37]:
v_sims = gensim.similarities.Similarity(outputs,tf_idf[v_corpus],num_features=len(v_dictionary))
print(v_sims)
print(type(v_sims))

Similarity index with 20 documents in 0 shards (stored under c:/outputs/)
<class 'gensim.similarities.docsim.Similarity'>


<strong>Tentado recuperar artigos dos periódicos em: The European Library</strong><br/>
URL: http://www.theeuropeanlibrary.org <br/>
Resultado: dados protegidos, mesmo com acesso autenticado pelo Minha-UFMG

In [38]:
# Buscar artigos de outra base de dados de periódicos e testar similaridade com o corpus montado para PubMED
# Foram encontrados 6 artigos com o termo OpenEHR no título mas não puderam ser recuperados
try:
   v_terbus2 = '(advanced(TITLE,OpenEHR))'
   v_url2    = url.request.urlopen('http://www.theeuropeanlibrary.org/tel4/search?query='+v_terbus2) 
   v_soup2   = bs(v_url2,'html.parser')
   #print(v_soup2)
   v_titEURO = [] # Declaração de um vetor para armazenar os títulos encontrados na 
                                                    
   # Os resultados da consulta estão organizados em uma tabela de tag <table>.
   # Para acessá-los nesta página precisaria acessar as tags <a> que me dariam os nomes dos artigos, porém elas
   # não estão visíveis, como se pode observar pelos resultados. 
   for v_elemento2 in v_soup2.find_all('table', class_='items'):  # tags html usadas para exibir os resultados da busca   
         print(v_elemento2)
        
except url.error:
   print('Falha na conexão!')
   exit(1) #interrompe  a execução do programa              
#print(v_titles)


<table class="items">
<div>
</div>
</table>


<strong>Tentado recuperar artigos dos periódicos em: DOAJ - Directory of Open Access Journal</strong><br/>
URL: https://doaj.org
Resultado: Solicita que o javascript esteja habilitado para acesso ao elemento de resultados da busca. Mesmo usando o
           navegador Mozilla Firefox com javascript.enabled = true, não consegui acesso.

In [48]:
# Buscar artigos de outra base de dados de periódicos e testar similaridade com o corpus montado para PubMED
# Base de periódicos escolhida: 
# Foram encontrados 6 artigos com o termo OpenEHR no título
try:
   v_terbus3 = '{"query":{"query_string":{"query":"openehr","default_field":"bibjson.title"}}}'
   v_url3    = url.request.urlopen('https://doaj.org/search?source='+v_terbus3) 
   v_soup3   = bs(v_url3,'html.parser')
   print(v_soup3)
   v_titDOAJ = [] # Declaração de um vetor para armazenar os títulos encontrados na 
                                                    
   # Os resultados na página de resultados estão organizados em uma tabela tag <table>
   # Para acessá-los nesta página precisaria acessar as tags <a> que me dariam os nomes dos artigos, porém elas
   # não estão visíveis, como se pode observar pelos resultados. Desta forma irei buscar outra base de periódicos.
   for v_elemento3 in v_soup3.find_all('div'):  # tags html usadas para exibir os resultados da busca   
         print(v_elemento3)
   # Detectei que a classe que preciso acessar é "facetview journals_and_articles", porém, preciso do JavaScript ativado
   # para acessar a área dos resultados e estou sem ele instalado. 
except url.error:
   print('Falha na conexão!')
   exit(1) #interrompe  a execução do programa              
#print(v_titles)


<!DOCTYPE html>

<html dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Directory of Open Access Journals</title>
<meta content="DOAJ is an online directory that indexes and provides access to quality open access, peer-reviewed journals." name="description"/>
<meta content="DOAJ" name="author"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var o=t[n]={exports:{}};e[n][0].call(o.exports,function(t){var o=e[n][1][t];return r(o||t)},o,o.exports)}return t[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(e,t,n){function r(){}function o(e,t,n){return function(){return i(e,[f.now()].concat(u(arguments)),t?null:this,n),t?void 0:this}}var i=e("handle"),a=e(2),u=e(3),c=e("ee").get("tracer"),f=e("loader"),s=NREUM;"undefined"==

In [55]:
# Vou realizar o teste de similaridade com os títulos dos artigos, coletados manualmente.
v_texto = "Modeling and Integration of Intensive Care Data Into an openEHR-based Enterprise Data Warehouse. "
v_texto += "Modeling healthcare authorization and claim submissions using the openEHR dual-model approach."
print(v_texto)
query_doc = [w.lower() for w in word_tokenize(v_texto)]
print(query_doc)
query_doc_bow = v_dictionary.doc2bow(query_doc)
print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)

Modeling and Integration of Intensive Care Data Into an openEHR-based Enterprise Data Warehouse. Modeling healthcare authorization and claim submissions using the openEHR dual-model approach.
['modeling', 'and', 'integration', 'of', 'intensive', 'care', 'data', 'into', 'an', 'openehr-based', 'enterprise', 'data', 'warehouse', '.', 'modeling', 'healthcare', 'authorization', 'and', 'claim', 'submissions', 'using', 'the', 'openehr', 'dual-model', 'approach', '.']
[(3, 2), (7, 1), (18, 2), (20, 1), (26, 1), (37, 2), (40, 1), (50, 1), (61, 1), (73, 2), (74, 1), (77, 1), (128, 1), (130, 1), (142, 1)]
[(3, 0.4327122897456187), (7, 0.21635614487280935), (18, 0.27402553286521125), (20, 0.21635614487280935), (26, 0.16629604666600473), (37, 0.33259209333200945), (40, 0.21635614487280935), (50, 0.21635614487280935), (61, 0.21635614487280935), (73, 0.4327122897456187), (74, 0.16629604666600473), (77, 0.10012019641360922), (128, 0.21635614487280935), (130, 0.21635614487280935), (142, 0.2163561448728

In [57]:
# Os resultados da In[55] e In[57] deverão ser melhor analisados. Ainda não consigo interpretá-los com segurança. 
v_sims[query_doc_tf_idf]

array([ 0.16758874,  0.11349124,  0.04005783,  0.17612736,  0.16607574,
        0.07770589,  0.06016717,  0.24611445,  0.07974071,  0.        ,
        0.        ,  0.        ,  0.01649406,  0.        ,  0.        ,
        0.15466206,  0.05710755,  0.1468644 ,  0.00983789,  0.01419657], dtype=float32)

<STRONG>OBSERVAÇÕES COLHIDAS DURANTE O DESENVOLVIMENTO e DEMAIS TESTES</STRONG>

In [43]:
# Este trecho é apenas para conferência sobre a tokenização em v_titles, sem eliminação das stopwords
# Utilizando a biblioteca nltk para obter as palavras dos títulos separadas em tokens
v_tokens = [[w.lower() for w in nltk.word_tokenize(text)] for text in v_titles]
print(v_tokens)
# Percebi que ao retirar as stopwords dos títulos, na In[96] o resultado já foi organizado em forma de tokens, não sendo
# portanto necessário realizar esta instrução para chegar no que eu pretendia. 

[['special', 'topic', 'interoperability', 'and', 'ehr', ':', 'combining', 'openehr', ',', 'snomed', ',', 'ihe', ',', 'and', 'continua', 'as', 'approaches', 'to', 'interoperability', 'on', 'national', 'ehealth', '.'], ['bridging', 'the', 'gap', 'between', 'the', 'social', 'and', 'the', 'technical', ':', 'the', 'enrolment', 'of', 'socio-technical', 'information', 'architects', 'to', 'cope', 'with', 'the', 'two-level', 'model', 'of', 'epr', 'systems', '.'], ['domain', 'modeling', 'and', 'application', 'development', 'of', 'an', 'archetype-', 'and', 'xml-based', 'ehrs', '.', 'practical', 'experiences', 'and', 'lessons', 'learnt', '.'], ['applying', 'an', 'archetype-based', 'approach', 'to', 'electroencephalography/event-related', 'potential', 'experiments', 'in', 'the', 'eegbase', 'resource', '.'], ['querying', 'archetype-based', 'electronic', 'health', 'records', 'using', 'hadoop', 'and', 'dewey', 'encoding', 'of', 'openehr', 'models', '.'], ['introducing', 'a', 'method', 'for', 'transfor

In [46]:
# Criação de um dicionário da lista dos documentos encontrados, baseado em seus títulos
# O dicionário atribui um número a cada termo encontrado. Este valor é útil para que se possa conferir a frequência de 
# ocorrência destes termos nos documentos.
v_dictionary = gensim.corpora.Dictionary(v_tokens)
print(v_dictionary[4])
print(v_dictionary.token2id['openehr'])
print("Number of words in dictionary:",len(v_dictionary))

for i in range(len(v_dictionary)):
    print(i, v_dictionary[i])

ehr
7
Number of words in dictionary: 184
0 special
1 topic
2 interoperability
3 and
4 ehr
5 :
6 combining
7 openehr
8 ,
9 snomed
10 ihe
11 continua
12 as
13 approaches
14 to
15 on
16 national
17 ehealth
18 .
19 bridging
20 the
21 gap
22 between
23 social
24 technical
25 enrolment
26 of
27 socio-technical
28 information
29 architects
30 cope
31 with
32 two-level
33 model
34 epr
35 systems
36 domain
37 modeling
38 application
39 development
40 an
41 archetype-
42 xml-based
43 ehrs
44 practical
45 experiences
46 lessons
47 learnt
48 applying
49 archetype-based
50 approach
51 electroencephalography/event-related
52 potential
53 experiments
54 in
55 eegbase
56 resource
57 querying
58 electronic
59 health
60 records
61 using
62 hadoop
63 dewey
64 encoding
65 models
66 introducing
67 a
68 method
69 for
70 transformation
71 paper-based
72 research
73 data
74 into
75 concept-based
76 representation
77 integration
78 hospital
79 clinical
80 decision
81 support
82 enable
83 reuse
84 record
85 str

In [47]:
# Teste de geração de nuvem de tags
from wordcloud import WordCloud
import matplotlib.pyplot as plt
 
wordcloud = WordCloud(max_font_size=100,width = 1520, height = 535).generate(text in v_titles)
plt.figure(figsize=(16,9))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

ModuleNotFoundError: No module named 'wordcloud'